In [1]:
!pip install -e ../.

Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///Users/mayanknautiyal/Desktop/Codes/CVAEI
  Preparing metadata (setup.py) ... done
  Running setup.py develop for cvaei


In [2]:
# %pip install --upgrade pip
# %pip install --force-reinstall "ray[tune]" hyperopt


In [3]:
import sys
sys.path.insert(0, '../')  


In [4]:
import cvaei
import torch
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from torch import nn, optim
from cvaei.examples.villar import Villar
from cvaei.models.conv_cvae import CNN_CVAE


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
print(torch.cuda.is_available())
print(torch.version.cuda)



Using device: cpu
False
None


In [5]:
villar = Villar()
train_theta_norm, train_data_norm, theta_normalizer, data_normalizer, val_theta_norm, val_data_norm = Villar.prepare_data(100)
observed_data = villar.observed_data()
#villar.check_normalizer()

2024-03-04 14:04:41,087 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2024-03-04 14:04:41,090 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2024-03-04 14:04:41,090 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2024-03-04 14:04:41,132 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2024-03-04 14:04:45,339 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2024-03-04 14:04:45,342 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2024-03-04 14:04:45,443 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2024-03-04 14:04:49,840 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2024-03-04 14:04:49,843 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2024-03-04 14:04:49,935 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2024-03-04 14:04:54,310 - GillesPy2 - WARNING - GillesPy2 simulation exceeded timeout.
2024-03-04 14:04:54,310 - GillesPy2 - WARNI

KeyboardInterrupt: 

In [ ]:
# train_theta_norm.shape, train_data_norm.shape

# Initialise the Model

In [ ]:
# gk_dist.plot_observation(train_data_norm, num_samples =1)
# gk_dist.plot_prior(train_theta_norm)

# Define the CVAE Network

In [ ]:
# Instantiate the CNN_CVAE model
input_dim = 15  # Example input dimension
latent_dim = 200  # Example latent space dimension
conditional_dim = 3  # Based on Nx3x200 conditional input
encoder_hidden_dims = [512,512]  # Example hidden dimensions for the encoder
sequence_length = 200  # Length of sequences in the conditional input
conv_output_channels = [64, 32]  # Channels in each Conv1D layer for the decoder
kernel_sizes = [3, 3]  # Kernel sizes for the Conv1D layers in the decoder
output_channels = 3  # Matching the original conditional input's channels



network = CNN_CVAE(
    input_dim=input_dim,
    latent_dim=latent_dim,
    conditional_dim=conditional_dim,
    encoder_hidden_dims=encoder_hidden_dims,
    decoder_hidden_dims=[],  # Assuming this is not used in the provided structure
    sequence_length=sequence_length,
    conv_output_channels=conv_output_channels,
    kernel_sizes=kernel_sizes,
    activation_fn= nn.LeakyReLU(0.1),
    w_recon=1.0,
    w_misfit=1.0,
    kld=1.0
)

optimizer = optim.AdamW(network.parameters(), lr=1e-3)

# train_dataset = TensorDataset(train_data_norm, train_theta_norm)
# val_dataset = TensorDataset(val_data_norm, val_theta_norm)

# train_loader = DataLoader(dataset=train_dataset, batch_size=128, shuffle=True)
# validation_loader = DataLoader(dataset=val_dataset, batch_size=128, shuffle=False)

print(network)

In [ ]:
network.train_model(train_loader=train_loader,
                    validation_loader= validation_loader,
                    optimizer=optimizer,
                    epochs=20,
                    num_cycles = 1,
                    theta_normalizer=theta_normalizer,
                    data_normalizer=data_normalizer,
                    
                    patience=100)

In [ ]:
network.plot_loss()

# Inference

In [ ]:
post = network.get_posterior(observed_data= observed_data, num_samples=10000)
post = theta_normalizer.inverse_transform(post)


In [ ]:
#gk_dist.plot_posterior(post)

In [ ]:
gk_dist.posterior_hist(post)